# Hespress Stories EDA

In [198]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import plotly.express as px

DATAPATH = Path("Data")
STORIES_FILES = [
    'stories_art-et-culture.csv',
    'stories_economie.csv',
    'stories_faits-divers.csv',
    'stories_marocains-du-monde.csv',
    'stories_medias.csv',
    'stories_orbites.csv',
    'stories_politique.csv',
    'stories_regions.csv',
    'stories_societe.csv',
    'stories_sport.csv',
    'stories_tamazight.csv'
]

## Read Data

In [144]:
dfs = []
for file in STORIES_FILES:
        df = pd.read_csv(DATAPATH / file, index_col=0)
        dfs.append(df)
stories_df = pd.concat(dfs, ignore_index=True)
stories_df.head()

,id,title,date,author,story,topic
0,f06aa998054e11eba66e646e69d991ea,"""بيت الشعر"" يسائل وزير الثقافة عن كوابيس سوداء",الجمعة 02 أكتوبر 2020 - 23:19,هسبريس من الرباط,"وجه ""بيت الشعر في المغرب"" إلى وزير الثقافة وال...",art-et-culture
1,f1cf1b9c054e11ebb718646e69d991ea,"مهرجان ""سينما المؤلّف"" يستحضر روح ثريا جبران",الجمعة 02 أكتوبر 2020 - 07:26,هسبريس من الرباط,في ظلّ استمرار حالة الطوارئ الصحية المرتبطة بج...,art-et-culture
2,f2d282a4054e11eb800f646e69d991ea,"فيلم ""بدون عنف"" لهشام العسري ..""كعب الحذاء ووا...",الجمعة 02 أكتوبر 2020 - 04:00,عفيفة الحسينات*,تشير مشاهدة فيلم قصير ضمن الثلاثية الأخيرة للم...,art-et-culture
3,f3f46cac054e11eba403646e69d991ea,"""تنين ووهان"" .. مريم أيت أحمد توقِّع أولى ""روا...",الجمعة 02 أكتوبر 2020 - 02:00,حاورَها: وائل بورشاشن,"مِن قَلب أيّام ""الحَجْر""، رأتِ النّورَ الفصول ...",art-et-culture
4,f50f0476054e11eba31b646e69d991ea,"مسكر يتخلّى عن دعم ""الوزارة"" بسبب ""الجمهور""",الخميس 01 أكتوبر 2020 - 19:40,هسبريس من الرباط,أعلن الفنان المغربيّ سعيد مسكر تخليه عن مبلغ ا...,art-et-culture


In [55]:
stories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11000 non-null  object
 1   title   11000 non-null  object
 2   date    11000 non-null  object
 3   author  11000 non-null  object
 4   story   11000 non-null  object
 5   topic   11000 non-null  object
dtypes: object(6)
memory usage: 515.8+ KB


## Data Processing

In [145]:
def arabic_date_parser(date):
    morocian_months_mapper = {
        'يناير': 'January',
        'فبراير': 'February',
        'مارس': 'March',
        'أبريل': 'April',
        'ماي': 'May',
        'يونيو': 'June',
        'يوليوز': 'July',
        'غشت': 'August',
        'شتنبر': 'September',
        'أكتوبر': 'October',
        'نونبر': 'November',
        'دجنبر': 'December'
    }

    date_pattern = r'(السبت|الأحد|الاثنين|الثلاثاء|الأربعاء|الخميس|الجمعة)\s(?P<day>\d{2})\s+(?P<month>يناير|فبراير|مارس|أبريل|ماي|يونيو|يوليوز|غشت|شتنبر|أكتوبر|نونبر|دجنبر)\s+(?P<year>\d{4})\s+-\s+(?P<time>\d{2}:\d{2})'

    match = re.match(date_pattern, date)
    day = match.group('day')

    month = match.group('month')
    month = morocian_months_mapper[month]
    year = match.group('year')
    time = match.group('time')
    parsed_date = day + ' ' + month + ' ' + year + ' - ' + time
    return parsed_date

In [146]:
stories_df["date"] = pd.to_datetime(stories_df["date"].apply(arabic_date_parser))
stories_df["day"] = stories_df["date"].dt.day.astype("category")
stories_df["month"] = stories_df["date"].dt.month.astype("category")
stories_df["year"] = stories_df["date"].dt.year.astype("category")
stories_df.head()

,id,title,date,author,story,topic,day,month,year
0,f06aa998054e11eba66e646e69d991ea,"""بيت الشعر"" يسائل وزير الثقافة عن كوابيس سوداء",2020-10-02 23:19:00,هسبريس من الرباط,"وجه ""بيت الشعر في المغرب"" إلى وزير الثقافة وال...",art-et-culture,2,10,2020
1,f1cf1b9c054e11ebb718646e69d991ea,"مهرجان ""سينما المؤلّف"" يستحضر روح ثريا جبران",2020-10-02 07:26:00,هسبريس من الرباط,في ظلّ استمرار حالة الطوارئ الصحية المرتبطة بج...,art-et-culture,2,10,2020
2,f2d282a4054e11eb800f646e69d991ea,"فيلم ""بدون عنف"" لهشام العسري ..""كعب الحذاء ووا...",2020-10-02 04:00:00,عفيفة الحسينات*,تشير مشاهدة فيلم قصير ضمن الثلاثية الأخيرة للم...,art-et-culture,2,10,2020
3,f3f46cac054e11eba403646e69d991ea,"""تنين ووهان"" .. مريم أيت أحمد توقِّع أولى ""روا...",2020-10-02 02:00:00,حاورَها: وائل بورشاشن,"مِن قَلب أيّام ""الحَجْر""، رأتِ النّورَ الفصول ...",art-et-culture,2,10,2020
4,f50f0476054e11eba31b646e69d991ea,"مسكر يتخلّى عن دعم ""الوزارة"" بسبب ""الجمهور""",2020-10-01 19:40:00,هسبريس من الرباط,أعلن الفنان المغربيّ سعيد مسكر تخليه عن مبلغ ا...,art-et-culture,1,10,2020


In [143]:
stories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      11000 non-null  object        
 1   title   11000 non-null  object        
 2   date    11000 non-null  datetime64[ns]
 3   author  11000 non-null  object        
 4   story   11000 non-null  object        
 5   topic   11000 non-null  object        
 6   day     11000 non-null  int64         
 7   month   11000 non-null  int64         
 8   year    11000 non-null  category      
dtypes: category(1), datetime64[ns](1), int64(2), object(5)
memory usage: 698.7+ KB


## EDA

### Stories count per topic

In [217]:
stories_df.groupby("topic").count()[["id"]].rename({"id": "count"}, axis=1)

| topic              |   count |
|:-------------------|--------:|
| art-et-culture     |    1000 |
| economie           |    1000 |
| faits-divers       |    1000 |
| marocains-du-monde |    1000 |
| medias             |    1000 |
| orbites            |    1000 |
| politique          |    1000 |
| regions            |    1000 |
| societe            |    1000 |
| sport              |    1000 |
| tamazight          |    1000 |


The data contains 11 diferent topics, exactly 1000 story example per topic.

### Stories count per topic and year

In [151]:
fig = px.bar(stories_df.groupby(["topic", "year"], as_index=False).count().rename({"id": "count"}, axis=1),
             x="topic", y="count", color='year', barmode='group', height=400)
fig.show()

Except for `['tamazight', 'medias', 'marocains-du-monde', 'art-et-culture']`, the rest of the topics will have the majority of their 1000 examples by 2020.

### Stories count per author and topic

In [176]:
active_authors = stories_df.groupby(["author"], as_index=False).count()
active_authors = active_authors[active_authors["id"]>100]["author"].values

authors_filtered_df = stories_df[stories_df["author"].isin(active_authors)]

fig = px.bar(authors_filtered_df.groupby(["author", "topic"], as_index=False).count().rename({"id": "count"}, axis=1),
             x="author", y="count", color='topic', barmode='group', height=500)
fig.show()

It is clear from the plot that famous authors do not write uniformly on various topics.

### Bigrams analysis

In [189]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(2, 2), max_features=100000).fit(stories_df["story"])
vectorizer.get_feature_names()

['00',
 '00 درهم',
 '000',
 '000 00',
 '000 درهم',
 '000 شخص',
 '000 منصب',
 '01',
 '02',
 '03',
 '04',
 '04 20',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '10 000',
 '10 آلاف',
 '10 أبريل',
 '10 أشخاص',
 '10 أكتوبر',
 '10 أيام',
 '10 إلى',
 '10 بالمائة',
 '10 حالات',
 '10 سنوات',
 '10 شتنبر',
 '10 غشت',
 '10 فبراير',
 '10 في',
 '10 لاعبين',
 '10 مارس',
 '10 ملايين',
 '10 مليارات',
 '10 من',
 '10 يوليوز',
 '10 يونيو',
 '100',
 '100 000',
 '100 ألف',
 '100 حالة',
 '100 درهم',
 '100 شخص',
 '100 في',
 '100 مدينة',
 '100 مليار',
 '100 مليون',
 '100 من',
 '100 يوم',
 '1000',
 '1000 درهم',
 '10000',
 '101',
 '102',
 '102 حالة',
 '103',
 '104',
 '104 12',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '11 ألف',
 '11 إلى',
 '11 حالة',
 '11 سنة',
 '11 شتنبر',
 '11 في',
 '11 مارس',
 '11 مباراة',
 '11 مليار',
 '11 مليون',
 '11 من',
 '11 يناير',
 '11 يوليوز',
 '11 يونيو',
 '110',
 '111',
 '112',
 '113',
 '113 13',
 '113 حالة',
 '114',
 '114 حالة',
 '115',
 '115 حالة',
 '116',
 '117',
 '118',
 '1

We can see that the numbers are dominating tokens results, we could simply define a token regex pattern `[ء-ي]+` to only capture arabic words instead of numbers.

In [195]:
vectorizer2 = CountVectorizer(ngram_range=(2, 2), max_features=100000, token_pattern=r'[ء-ي]+').fit(stories_df["story"])
vectorizer2.get_feature_names()

['آباء وأمهات',
 'آباء وأولياء',
 'آتية من',
 'آثار أزمة',
 'آثار الأزمة',
 'آثار الجائحة',
 'آثار الجفاف',
 'آثار جائحة',
 'آثار سلبية',
 'آثار هذه',
 'آثارا سلبية',
 'آثارها السلبية',
 'آثارها على',
 'آجال الأداء',
 'آجال معقولة',
 'آجال مناسبة',
 'آجالها القانونية',
 'آخ ر',
 'آخذة في',
 'آخر أجل',
 'آخر أشارت',
 'آخر أعمال',
 'آخر أفادت',
 'آخر أن',
 'آخر أوردت',
 'آخر أي',
 'آخر إلى',
 'آخر إن',
 'آخر اجتماع',
 'آخر الإحصائيات',
 'آخر اللمسات',
 'آخر المطاف',
 'آخر المعطيات',
 'آخر تحديث',
 'آخر تطورات',
 'آخر حول',
 'آخر حيث',
 'آخر ذكر',
 'آخر ذكرت',
 'آخر رحلاتها',
 'آخر رمق',
 'آخر زيارة',
 'آخر ساعة',
 'آخر سنة',
 'آخر سواء',
 'آخر سوى',
 'آخر ضمن',
 'آخر على',
 'آخر عن',
 'آخر غير',
 'آخر فإن',
 'آخر في',
 'آخر قال',
 'آخر قالت',
 'آخر كان',
 'آخر كانت',
 'آخر كتبت',
 'آخر كما',
 'آخر لأن',
 'آخر لا',
 'آخر لحظة',
 'آخر لقاء',
 'آخر له',
 'آخر ما',
 'آخر مباراة',
 'آخر مذكرة',
 'آخر مرة',
 'آخر مع',
 'آخر من',
 'آخر موعد',
 'آخر نشرت',
 'آخر هذه',
 'آخر هو',
 'آخر وأضافت',
 

In [212]:
X = vectorizer2.transform(stories_df["story"])
bigrams_df = pd.DataFrame()
bigrams_df["Bigrams"] = vectorizer2.get_feature_names()
bigrams_df["Count"] = np.ravel(np.sum(X, axis=0))

In [218]:
bigrams_df.head(10)

|    | Bigrams      |   Count |
|---:|:-------------|--------:|
|  0 | آباء وأمهات  |      50 |
|  1 | آباء وأولياء |      46 |
|  2 | آتية من      |      12 |
|  3 | آثار أزمة    |      10 |
|  4 | آثار الأزمة  |      15 |
|  5 | آثار الجائحة |      14 |
|  6 | آثار الجفاف  |      10 |
|  7 | آثار جائحة   |      19 |
|  8 | آثار سلبية   |      12 |
|  9 | آثار هذه     |       7 |


We can see the count of bigrams occurence in stories.